.h2 Analyze weather data
First load measurements from export

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal
import torch

ModuleNotFoundError: No module named 'pandas'

In [ ]:
raw_data = pd.read_csv("~/Desktop/produkt_klima_tag_18790101_20201231_04104.txt", "\t", skipinitialspace=True)
raw_data = raw_data[18261:]
raw_data["MESS_DATUM"] = pd.to_datetime(raw_data["MESS_DATUM"], format="%Y%m%d")
raw_data["DAY_OF_YEAR"] = raw_data["MESS_DATUM"].dt.day_of_year
raw_data = raw_data.drop(labels=["STATIONS_ID"], axis=1)
raw_data = raw_data.drop(labels=["FX"], axis=1) #  Windgeschwindigkeit
raw_data = raw_data.drop(labels=["FM"], axis=1) #  Windgeschwindigkeit

rainfall = raw_data["RSK"]
sunhours = raw_data["SDK"]

In [ ]:
res = plt.hist(raw_data["RSK"], 100, (0., 10.)), plt.ylim(0, 1000), plt.show()

mean_rain = raw_data.groupby("DAY_OF_YEAR")["RSK"].mean()
plt.plot(mean_rain), plt.plot(scipy.signal.convolve(mean_rain[:-1] / 15., np.ones(15))), plt.show()

rainy_days = raw_data[raw_data["RSK"] > 1.]
some_rain = rainy_days.groupby("DAY_OF_YEAR")["RSK"].count() / 61.
plt.plot(some_rain), plt.plot(scipy.signal.medfilt(some_rain[:-1], 15)), plt.show()
plt.plot(some_rain), plt.plot(scipy.signal.convolve(some_rain[:-1] / 15., np.ones(15))), plt.show()

sun_hours_median = raw_data.groupby("DAY_OF_YEAR")["SDK"].median()
sun_hours_mean = raw_data.groupby("DAY_OF_YEAR")["SDK"].mean()
plt.plot(sun_hours_mean), plt.plot(sun_hours_median), plt.show()
plt.plot(sun_hours_median), plt.show()

sunny_days = raw_data[raw_data["SDK"] > 6]
sunny_days_chance = sunny_days.groupby("DAY_OF_YEAR")["SDK"].count() / 61.
plt.plot(sunny_days_chance), plt.show()


plt.plot(some_rain), plt.plot(sunny_days_chance), plt.show()





import torch

dtype = torch.float
device = torch.device("cpu")


def l1(y, y_pred, w):
    return (w * (y_pred - y)).abs().pow(1.1).mean()


def l1_d_reg(y_pred, x):
    dif_y = y_pred[1:] - y_pred[:-1]
    dif_x = x[1:] - x[:-1]
    steigung = dif_y / dif_x
    return steigung.abs().pow(1.1).mean()


def l2_dd_reg(y_pred, x):
    dif_y = y_pred[1:-1] - y_pred[2:] - y_pred[:-2]
    dif_x = x[2:] - x[:-2]
    steigung = dif_y / dif_x
    return steigung.abs().pow(1.1).mean()


def model(measurement, timestamp, weights, rs, lr, model="l1", reg="l2_dd"):
    x = torch.tensor(timestamp, device=device, dtype=dtype)
    y = torch.tensor(measurement, device=device, dtype=dtype)
    w = torch.tensor(weights, device=device, dtype=dtype)

    y_pred = torch.tensor(y, device=device, dtype=dtype, requires_grad=True)

    history = []
    for i in range(10001):
        loss = l1(y, y_pred, w) + rs * l2_dd_reg(y_pred, x)
        if i % 100 == 0:
            print(i, loss.item())
            history.append(loss.item())
        loss.backward()
        with torch.no_grad():
            y_pred -= lr * y_pred.grad
            y_pred.grad = None

    return y_pred, history


def model_2(measurement, timestamp, weights, rs, lr, model="l1", reg="l2_dd"):
    x = torch.tensor(timestamp, device=device, dtype=dtype)
    y = torch.tensor(measurement, device=device, dtype=dtype)
    w = torch.tensor(weights, device=device, dtype=dtype)

    y_pred = torch.tensor(y, device=device, dtype=dtype, requires_grad=True)

    maeLoss = torch.nn.L1Loss()

    d_y_l = (y_pred[0:-2] - y_pred[1:-1]) / (x[0:-2] - x[1:-1])
    d_y_r = y_pred[1:-1] - y_pred[2:] / (x[1:-1] - x[2:])
    dd_y = (d_y_l - d_y_r) / (0.5 * (x[0:-2] + x[1:-1]) - 0.5 * (x[1:-1] + x[2:]))

    grad = ((w * (y_pred - y)).abs().pow(1.1).mean() + rs * dd_y.abs().pow(1.1).mean()).backward()

    opt = torch.optim.RMSprop([y_pred], lr=lr)

    for i in range(10001):
        def closure():
            opt.zero_grad()
            output = y_pred
            loss = l1(y, y_pred, w) + rs * l2_dd_reg(y_pred, x)
            if i % 100 == 0:
                print(i, loss.item())
            loss.backward()
            return loss

        opt.step(closure)


y_pred_final = y_pred.detach().numpy() * measurements_std + measurements_mean
y_final = y.detach().numpy() * measurements_std + measurements_mean



In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

data = np.asarray(data)
# Data ß - some key
plt.figure(), plt.plot(np.int32(data[:,0]))#%%
# Data 1 - always 1
plt.figure(), plt.plot(np.int32(data[:,1]))#%%
# Data 2 - "deleted"
data_deleted = np.int32(data[:,2])
deleted_mask = (np.asarray(data_deleted, dtype=np.bool))
plt.figure(), plt.plot(data_deleted)#%%
# Data 3 - date
plt.figure(), plt.plot(np.int32(data[:,3]))#%%
timestamps = np.int64(data[:,3])
measurementTime = []
for timestamp in timestamps:
    measurementTime.append(datetime.utcfromtimestamp(timestamp/1000).strftime('%Y-%m-%d %H:%M:%S'))
# Data 4 - weight reading
weightMeasurements = np.float64(data[:,4])
plt.figure(), plt.plot(weightMeasurements)

weightMeasurements_filtered = weightMeasurements[deleted_mask]
weight_mean = weightMeasurements_filtered.mean()
weight_std = weightMeasurements_filtered.std()
weightMeasurements_filtered -= weight_mean
weightMeasurements_filtered /= weight_std
timestamps_filtered = timestamps[deleted_mask]
timestamps_filtered -= timestamps_filtered[0]
timestamps_filtered = np.asarray(timestamps_filtered, dtype=np.float32)
timestamps_filtered /= np.float(timestamps_filtered[-1])

weights = np.ones_like(timestamps_filtered)
for i in range(len(weights)):
    weights[i]=1./(np.sum(np.abs(timestamps_filtered-timestamps_filtered[i])<0.06))

plt.figure(), plt.plot(timestamps_filtered, weightMeasurements_filtered)
plt.figure(), plt.plot(timestamps_filtered, weights)

In [ ]:
import scipy.optimize

import numpy as np

def l2(y, y_mod, w):
    return np.mean((y-y_mod)**2*w)

def l1(y, y_mod, w):
    return np.mean(np.abs(y-y_mod)**1.1*w)

def l1_unequal(y, y_mod, w):
    dif = y-y_mod
    dif[dif<0] *= 5.
    dif = np.abs(dif)**1.1
    return np.mean(dif*w)

def l2_reg(y_mod, x):
    dif_y = y_mod[1:] - y_mod[:-1]
    dif_x = x[1:] - x[:-1]
    steigung = dif_y / dif_x
    #print(steigung)
    return np.mean(steigung**2)

def l1_reg(y_mod, x):
    dif_y = y_mod[1:] - y_mod[:-1]
    dif_x = x[1:] - x[:-1]
    steigung = dif_y / dif_x
    #print(steigung)
    return np.mean(np.abs(steigung)**1.1)

def costfunction(y_mod, x, y, w, r):
    return l1_unequal(y, y_mod, w) + r * l2_reg(y_mod, x)

y_res = scipy.optimize.minimize(
    costfunction,
    weightMeasurements_filtered.copy(),
    method='BFGS',
    options={
        'maxiter': 100000
    },
    args=(timestamps_filtered
          , weightMeasurements_filtered, weights, 10**-2)
)
print(y_res.message)

plt.figure(), plt.plot(timestamps_filtered, weightMeasurements_filtered*weight_std+weight_mean,'x'), plt.plot(timestamps_filtered, y_res.x*weight_std+weight_mean)